In [ ]:
#cleaning the existing csv files using pandas

import pandas as pd

# Veri setlerini yükle
# Not: Kullanıcı 'exercises.csv.xlsx - exercises.csv' dosyasını yüklemişti.
df_kaggle = pd.read_excel("data/database/exercises.csv.xlsx")

# 1. Instructions (Talimatlar) sütunlarını birleştirme
instruction_cols = [col for col in df_kaggle.columns if 'instructions' in col]
# Her satır için dolu olan instruction sütunlarını birleştiriyoruz.
df_kaggle['description'] = df_kaggle[instruction_cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# 2. Secondary Muscles (İkincil Kaslar) sütunlarını birleştirme
secondary_cols = [col for col in df_kaggle.columns if 'secondaryMuscles' in col]
df_kaggle['secondary_muscles_combined'] = df_kaggle[secondary_cols].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# 3. Yeni DataFrame oluşturma (Kullanıcının şablonuna uygun)
new_df = pd.DataFrame()

new_df['id'] = df_kaggle['id']
new_df['name'] = df_kaggle['name']
# Kullanıcının isteği üzerine 'target' sütununu 'primary_muscles' olarak alıyoruz.
new_df['primary_muscles'] = df_kaggle['target'] 
new_df['secondary_muscles'] = df_kaggle['secondary_muscles_combined']
new_df['body_part'] = df_kaggle['bodyPart'] # Filtreleme için yararlı olabilir diye tutuyoruz
new_df['equipment'] = df_kaggle['equipment']
new_df['difficulty'] = None # Doldurulacak
new_df['risk_level'] = None # Doldurulacak
new_df['impact'] = None # Doldurulacak
new_df['suitable_for'] = None # Doldurulacak
new_df['!suitable_for'] = None # Doldurulacak (Kritik)
new_df['description'] = df_kaggle['description']
#new_df['gif_url'] = df_kaggle['gifUrl']

# 4. CSV olarak kaydetme
output_filename = 'exercises_cleaned.csv'
new_df.to_csv(output_filename, index=False)

print(f"Dosya '{output_filename}' adıyla oluşturuldu.")
print(new_df.head().to_markdown(index=False, numalign="left", stralign="left"))

Dosya 'exercises_cleaned.csv' adıyla oluşturuldu.
| id   | name                    | primary_muscles   | secondary_muscles       | body_part   | equipment   | difficulty   | risk_level   | impact   | suitable_for   | !suitable_for   | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|:-----|:------------------------|:------------------|:------------------------|:------------|:------------|:-------------|:-------------|:---------|:---------------|:----------------|:-------------------------------------------------------------------------------------------------

In [3]:
import sys
sys.executable

'c:\\Users\\odz-2\\anaconda3\\envs\\excbot\\python.exe'

In [5]:
%pip install pandas


   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.3 MB 10.0 MB/s eta 0:00:01
   ------------ --------------------------- 3.4/11.3 MB 9.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.9/11.3 MB 6.7 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.3 MB 5.5 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.3 MB 5.0 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.3 MB 5.0 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.3 MB 3.5 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.3 MB 3.3 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.3 MB 3.4 MB/s eta 0:00:02
   ---------------------------- ----------- 8.1/11.3 MB 3.8 MB/s eta 0:00:01
   ------------------------------- -------- 8.9/11.3 MB 3.8 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.3 MB 3.7 MB/s eta 0:00:01
   --

In [13]:
import os
os.chdir("../")

In [14]:
%pwd

'c:\\Users\\odz-2\\Desktop\\itü\\bitirme\\2nd part\\exercise-chatbot-project\\exercise-chatbot-project'

In [17]:
%pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

df = pd.read_csv("data/database/exercises_cleaned.csv")

def refine_difficulty(row):
    name = row['name'].lower()
    equipment = row['equipment']
    
    # --- 1. Adım: Kelime Bazlı "Zor" Hareketler ---
    # Bu kelimeler geçiyorsa kesinlikle ZORDUR (Advanced)
    hard_keywords = ['pull-up', 'chin-up', 'muscle-up', 'dip', 'dips', 'handstand', 'pistol', 'clean', 'snatch']
    if any(word in name for word in hard_keywords):
        return 'Advanced'

    # --- 2. Adım: Kelime Bazlı "Kolay" Hareketler ---
    # Bu kelimeler geçiyorsa genelde KOLAYDIR (Beginner)
    easy_keywords = ['stretch', 'wall', 'kneeling', 'march']
    if any(word in name for word in easy_keywords):
        return 'Beginner'

    # --- 3. Adım: Geriye Kalanlar İçin Ekipman Kontrolü ---
    if equipment == 'leverage machine' | 'band' | 'cable' | 'resistance band' | 'roller' | 'sled machine' | 'assisted' | 'medicine ball':
        return 'Beginner'
    elif equipment == 'dumbbell' | 'barbell' | 'ez barbell' | 'kettlebell':
        return 'Intermediate' # Barbell ama clean/snatch değilse ortadır
    elif equipment == 'Body Only':
        return 'Intermediate' # Barfiks değilse, squat/lunge gibi ortadır
    else:
        return 'Intermediate'

# Fonksiyonu Uygula
df['difficulty'] = df.apply(refine_difficulty, axis=1)

output_filename = 'exercises_cleaned1.csv'
new_df.to_csv(output_filename, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/database/exercises_cleaned.csv'